In [11]:
import os
import tensorflow as tf
import pandas as pd


In [12]:

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a floast_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array


def parse_tfr_element(element):
    #use the same structure as above; it's kinda an outline of the structure we now want to create
    data = {
        'emb' : tf.io.FixedLenFeature([], tf.string),
        'label' : tf.io.FixedLenFeature([], tf.int64)
    }


    content = tf.io.parse_single_example(element, data)

    label = content['label']
    emb = content['emb']


    #get our 'feature'-- our image -- and reshape it appropriately
    feature = tf.io.parse_tensor(emb, out_type=tf.float32)
    return (feature, label)


def get_dataset(filename):
    #create the dataset
    dataset = tf.data.TFRecordDataset(filename)

    #pass every single feature through our mapping function
    dataset = dataset.map(
        parse_tfr_element
    )

    return dataset




In [13]:
args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "embeddings":"music_style",
    "train_id": "global_sample_test"
})

In [14]:
train_id = "global_sample_test"

In [15]:
sample_size = 0.1 ## 10% do conjunto de dados

In [16]:
job_path = "/mnt/disks/data/fma/trains"

In [17]:
train_path = os.path.join(job_path,train_id)

In [25]:
files = os.path.join(train_path,'tfrecords','validation')

In [26]:
os.listdir(files)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/disks/data/fma/trains/global_sample_test/tfrecords/validation'

In [19]:
dataset = get_dataset(files)

for sample in dataset.take(1):
    print(sample[0].shape)
    print(sample[1].shape)
    print(sample)

2023-01-30 12:34:20.207068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-30 12:34:20.875594: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9621 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} /mnt/disks/data/fma/trains/tfrecords/validation; No such file or directory [Op:IteratorGetNext]